In [ ]:
pip install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import padding

In [ ]:
def generate_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password)
    return key

In [ ]:
def encrypt_file(file_path, password):
    salt = b'\x86v\xc8\xee\xef\x98\xf0T\xc9c\xf8P\x90\xb1\x85'
    key = generate_key(password, salt)

    iv = b'\x00' * 12  # Initialization Vector (IV) for AES-GCM mode
    aad = None  # Additional Associated Data (optional)

    with open(file_path, 'rb') as file:
        plaintext = file.read()

    # Perform AES-GCM encryption
    encryptor = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend()).encryptor()
    encryptor.authenticate_additional_data(aad)
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    tag = encryptor.tag

    return ciphertext, salt, iv, tag

In [ ]:
def decrypt_file(ciphertext, salt, iv, tag, password):
    key = generate_key(password, salt)

    decryptor = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend()).decryptor()
    decryptor.authenticate_additional_data(None)
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    return plaintext

In [ ]:
def test_encryption_decryption():
    file_path = 'path/to/your/file.txt'
    password = b'your_password'

    # Encrypt the file
    ciphertext, salt, iv, tag = encrypt_file(file_path, password)

    # Save the encrypted file to disk
    with open('encrypted_file.bin', 'wb') as file:
        file.write(ciphertext)

    # Decrypt the file
    decrypted_text = decrypt_file(ciphertext, salt, iv, tag, password)

    # Save the decrypted file to disk
    with open('decrypted_file.txt', 'wb') as file:
        file.write(decrypted_text)

    # Verify if the decrypted file matches the original file
    with open(file_path, 'rb') as file:
        original_text = file.read()

    assert decrypted_text == original_text
    print